In [29]:
import netCDF4
import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta
from tqdm import tqdm
from shapely.geometry import Point
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, entropy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.fft import fft
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
import os
import os
from pycaret.classification import *

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches


In [30]:

class SurfaceTypeUtils:
    surface_type_dict = {
        -1: "Ocean",
        0: "NaN",
        1: "Artifical",
        2: "Barely vegetated",
        3: "Inland water",
        4: "Crop",
        5: "Grass",
        6: "Shrub",
        7: "Forest"
    }
    ddm_antennas = {
        0: 'None',
        1: 'Zenith',
        2: 'LHCP',
        3: 'RHCP',
    }


In [31]:

class GeoUtils:
    def __init__(self, world_shapefile_path):
        self.world = gpd.read_file(world_shapefile_path)

    @staticmethod
    def add_seconds(time, seconds):
        timestamp = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        new_timestamp = timestamp + timedelta(seconds=seconds)
        return new_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    def is_land(self, lat, lon):
        point = Point(lon, lat)
        return any(self.world.contains(point))

    @staticmethod
    def check_ocean_and_land(lst):
        has_ocean = -1 in lst
        has_land = any(1 <= num <= 7 for num in lst)
        return has_ocean and has_land

    @staticmethod
    def fill_and_filter(arr):
        mask_all_nan = np.all(np.isnan(arr), axis=(2, 3))
        arr_filled = arr.copy()
        for i in range(arr.shape[0]):
            nan_indices = np.where(mask_all_nan[i])[0]
            if len(nan_indices) > 0:
                valid_indices = np.where(~mask_all_nan[i])[0]
                if len(valid_indices) > 0:
                    mean_matrix = np.nanmean(arr[i, valid_indices, :, :], axis=0)
                    arr_filled[i, nan_indices, :, :] = mean_matrix
        mask_discard = np.all(mask_all_nan, axis=1)
        arr_filtered = arr_filled[~mask_discard]
        return arr_filtered, list(np.where(mask_discard.astype(int) == 1)[0])


In [32]:

class NetCDFPreprocessor:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.netcdf_file_list = os.listdir(root_dir)

    def preprocess(self, f):
        raw_counts = np.array(f.variables['raw_counts'])
        # Calcolo distanza tra il punto speculare e l'aereo
        ac_alt_2d = np.repeat(np.array(f.variables['ac_alt'])[:, np.newaxis], 20, axis=1)
        distance_2d = (ac_alt_2d - f.variables['sp_alt'][:]) / np.cos(np.deg2rad(f.variables['sp_inc_angle'][:]))

        # Seleziono gli indici dove sp_rx_gain_copol > 5, sp_rx_gain_xpol > 5 e ddm_snr > 0 e distanza tra punto speculare e antenna > 2000 e < 10000

        copol = f.variables['sp_rx_gain_copol'][:]
        xpol = f.variables['sp_rx_gain_xpol'][:]
        snr = f.variables['ddm_snr'][:]
        dist = distance_2d[:]
        
        keep_mask = (copol >= 5) & (xpol >= 5) & (snr > 0) & ((dist >= 2000) & (dist <= 10000)) & (~np.isnan(copol.data) & ~np.isnan(xpol.data) & ~np.isnan(snr.data) & ~np.isnan(dist.data))
        to_keep_indices = np.argwhere(keep_mask)
        filtered_raw_counts = [raw_counts[i, j] for i, j in to_keep_indices]
        output_array = np.full(raw_counts.shape, np.nan, dtype=np.float32)

        for idx, (i, j) in enumerate(to_keep_indices):
            output_array[i, j] = filtered_raw_counts[idx]
        raw_counts_filtered = output_array.copy()

        del output_array

        ddm_data_dict = {
            'Raw_Counts': raw_counts_filtered.reshape(raw_counts_filtered.shape[0]*raw_counts_filtered.shape[1], raw_counts_filtered.shape[2], raw_counts_filtered.shape[3]),
        }
        keep_indices = np.where(
            np.all(~np.isnan(ddm_data_dict['Raw_Counts']), axis=(1, 2)) & (np.sum(ddm_data_dict['Raw_Counts'], axis=(1, 2)) > 0)
        )[0]
        fit_data = np.array([ddm_data_dict['Raw_Counts'][f].ravel() for f in keep_indices])
        surface_types = f.variables["sp_surface_type"][:]
        surface_types = np.nan_to_num(surface_types, nan=0)
        surface_types_unravelled = surface_types.ravel()
        label_data = [1 if surface_type in np.arange(1, 8) else 0 for surface_type in surface_types_unravelled]
        label_data = [label_data[l] for l in range(len(label_data)) if l in keep_indices]
        return fit_data, label_data


    def process_all_files(self, save = bool):
        
        from sklearn.model_selection import train_test_split
        full_data = []
        full_labels = []
        counter = 0
        for file_name in tqdm(self.netcdf_file_list, desc="Processing files"):
            if not file_name.endswith('.nc'):
                continue
            f = netCDF4.Dataset(f'{self.root_dir}{file_name}')
            data, labels = self.preprocess(f)
            full_data.append(data)
            full_labels.append(labels)
            counter += 1
            if counter == 70:
                break  # Limita il numero di file processati per test
                
        # Trova gli indici degli elementi di full_data con seconda dimensione uguale a 200
        valid_indices = [i for i, arr in enumerate(full_data) if arr.ndim == 2 if arr.shape[1] == 200]

        # Applica la selezione a full_data e full_labels
        full_data_clean = [full_data[i] for i in valid_indices]
        full_labels_clean = [full_labels[i] for i in valid_indices]

        #full_data_clean = np.vstack(full_data_clean) Unable to allocate memory for this operation
        #full_labels_clean = np.hstack(full_labels_clean) Unable to allocate memory for this operation
        #print(f"Shape of full_data_clean before chunking and sampling: {np.array(full_data_clean).shape}")
        #print(f"Shape of full_labels_clean before chunking and sampling: {np.array(full_labels_clean).shape}")
        
        # Chunking 
        
        import pyarrow as pa
        import pyarrow.parquet as pq
        from sklearn.model_selection import train_test_split
        os.makedirs('processed_data/binary_classification', exist_ok=True)

        chunk_size = 250 # dimensione del chunk in numero di campioni
        sample_fraction = 0.05  # frazione di dati da campionare per ogni chunk

        full_data_sampled = []
        full_labels_sampled = []

        num_chunks = int(np.ceil(len(full_data_clean) / chunk_size))
        for idx in range(num_chunks):
            start = idx * chunk_size
            end = min((idx + 1) * chunk_size, len(full_data_clean))
            chunk_data = np.vstack(full_data_clean[start:end])
            chunk_labels = np.vstack(full_labels_clean[start:end])
            
            print(f"Chunk {idx + 1}/{num_chunks} processed with shape {chunk_data.shape} and labels shape {chunk_labels.shape}")

            # Salva ogni chunk come file parquet separato
            fit_data_df = pd.DataFrame(chunk_data)
            labels_df = pd.DataFrame(chunk_labels, columns=['label'])

            table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
            table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)

            pq.write_table(
                table_fit,
                f'processed_data/binary_classification/fit_data_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )
            pq.write_table(
                table_labels,
                f'processed_data/binary_classification/labels_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )

        # Imposta la frazione di dati da campionare per ogni chunk (es: 0.2 per il 20%)
        
            _, X_sampled, _, y_sampled = train_test_split(
                chunk_data, chunk_labels, 
                test_size=sample_fraction, 
                stratify=chunk_labels, 
                random_state=42
            )
            del chunk_data, chunk_labels, table_fit, table_labels  # Libera memoria
            
            full_data_sampled.append(X_sampled)
            full_labels_sampled.append(y_sampled)

        full_data_sampled_stratified = np.vstack(full_data_sampled)
        full_labels_sampled_stratified = np.hstack(full_labels_sampled)
        print(f"Shape of sampled data after chunking and sampling: {np.array(full_data_sampled_stratified).shape}")
        print(f"Shape of sampled labels after chunking and sampling: {np.array(full_labels_sampled_stratified).shape}")
        
        # Crea la cartella processed_data se non esiste
        os.makedirs('processed_data/binary_classification', exist_ok=True)

        # Salva fit_data in formato parquet ottimizzato
        fit_data_df = pd.DataFrame(full_data_sampled_stratified)
        table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
        pq.write_table(
            table_fit,
            'processed_data/binary_classification/fit_data_stratified_binary.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        # Salva labels in formato parquet ottimizzato
        labels_df = pd.DataFrame(full_labels_sampled_stratified, columns=['label'])
        table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)
        pq.write_table(
            table_labels,
            'processed_data/binary_classification/labels_stratified_binary.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        return full_data_sampled_stratified, full_labels_sampled_stratified
    
    


In [33]:

class DDMFeatureExtractor:
    def __init__(self):
        pass

    def extract_ddm_features(self, ddm_array):
        """
        Estrae feature statistiche e fisiche da una DDM (Delay Doppler Map).
        
        Args:
            ddm_array (np.ndarray): Array 2D (delay x doppler) della DDM.
            
        Returns:
            dict: Dizionario di feature estratte.
        """
        features = {}
        # Feature statistiche di base
        features['ddm_mean'] = np.nanmean(ddm_array)
        features['ddm_std'] = np.nanstd(ddm_array)
        features['ddm_max'] = np.nanmax(ddm_array)
        features['ddm_min'] = np.nanmin(ddm_array)
        features['ddm_sum'] = np.nansum(ddm_array)
        features['ddm_median'] = np.nanmedian(ddm_array)
        features['ddm_skew'] = pd.Series(ddm_array.ravel()).skew()
        features['ddm_kurtosis'] = pd.Series(ddm_array.ravel()).kurtosis()
        
        # Feature fisiche: posizione del massimo (picco)
        max_idx = np.unravel_index(np.nanargmax(ddm_array), ddm_array.shape)
        features['ddm_peak_delay'] = max_idx[0]
        features['ddm_peak_doppler'] = max_idx[1]
        
        # Energia normalizzata
        norm_ddm = ddm_array / (np.nansum(ddm_array) + 1e-8)
        features['ddm_entropy'] = -np.nansum(norm_ddm * np.log(norm_ddm + 1e-8))
        
        # Rapporto picco/fondo
        background = np.delete(ddm_array.ravel(), np.nanargmax(ddm_array))
        features['ddm_peak_to_mean'] = features['ddm_max'] / (np.nanmean(background) + 1e-8)
        
        # Momenti centrali (2° e 3° ordine)
        delay_axis = np.arange(ddm_array.shape[0])
        doppler_axis = np.arange(ddm_array.shape[1])
        ddm_sum = np.nansum(ddm_array)
        if ddm_sum > 0:
            delay_mean = np.nansum(delay_axis[:, None] * ddm_array) / ddm_sum
            doppler_mean = np.nansum(doppler_axis[None, :] * ddm_array) / ddm_sum
            features['ddm_delay_mean'] = delay_mean
            features['ddm_doppler_mean'] = doppler_mean
            features['ddm_delay_var'] = np.nansum(((delay_axis[:, None] - delay_mean) ** 2) * ddm_array) / ddm_sum
            features['ddm_doppler_var'] = np.nansum(((doppler_axis[None, :] - doppler_mean) ** 2) * ddm_array) / ddm_sum
        else:
            features['ddm_delay_mean'] = np.nan
            features['ddm_doppler_mean'] = np.nan
            features['ddm_delay_var'] = np.nan
            features['ddm_doppler_var'] = np.nan

        return features

    def extract_features_from_dataset(self, fit_data):
        """
        Estrae feature ingegnerizzate da un dataset di DDM.
        
        Args:
            fit_data (np.ndarray): Array 2D (n_samples x 200) delle DDM flattenate.
            
        Returns:
            pd.DataFrame: DataFrame con le feature estratte.
        """
        n_delay = 20
        n_doppler = 10
        feature_list = []
        for ddm_flat in tqdm(fit_data, desc="Extracting DDM features"):
            ddm = ddm_flat.reshape(n_delay, n_doppler)
            features = self.extract_ddm_features(ddm)
            feature_list.append(features)
        return pd.DataFrame(feature_list)
    
    def create_ddm_features_MORE(self, fit_data: np.ndarray) -> pd.DataFrame:
        

        """
        Estrae features dettagliate da raw_counts DDM (shape: n_samples x 200)
        """

        def gini(array):
            """Calcola il coefficiente di Gini (disuguaglianza)"""
            array = np.sort(array)
            index = np.arange(1, array.shape[0] + 1)
            return (np.sum((2 * index - array.shape[0] - 1) * array)) / (array.shape[0] * np.sum(array))
        

        features = []

        for row in tqdm(fit_data, desc="Extracting DDM features"):
            f = {}
            x = np.array(row, dtype=np.float32) + 1e-10  # evita log(0)

            # 1. Statistiche base
            f['mean'] = np.mean(x)
            f['std'] = np.std(x)
            f['min'] = np.min(x)
            f['max'] = np.max(x)
            f['median'] = np.median(x)
            f['range'] = np.max(x) - np.min(x)
            f['skew'] = skew(x)
            f['kurtosis'] = kurtosis(x)
            f['entropy'] = entropy(x)
            f['gini'] = gini(x)

            # 2. Posizionali
            f['peak_index'] = np.argmax(x)
            f['peak_value'] = np.max(x)
            f['center_of_mass'] = np.sum(np.arange(len(x)) * x) / np.sum(x)
            f['inertia'] = np.sum(((np.arange(len(x)) - f['center_of_mass'])**2) * x)

            # 3. Segmentazione
            thirds = np.array_split(x, 3)
            for i, part in enumerate(thirds):
                f[f'sum_third_{i+1}'] = np.sum(part)
                f[f'mean_third_{i+1}'] = np.mean(part)
                f[f'max_third_{i+1}'] = np.max(part)
            
            windows = np.array_split(x, 5)
            for i, w in enumerate(windows):
                f[f'mean_w{i+1}'] = np.mean(w)
                f[f'std_w{i+1}'] = np.std(w)
                f[f'max_w{i+1}'] = np.max(w)

            # 4. Derivate e cambiamenti
            dx = np.diff(x)
            f['mean_diff'] = np.mean(dx)
            f['std_diff'] = np.std(dx)
            f['max_diff'] = np.max(dx)
            f['min_diff'] = np.min(dx)
            f['n_positive_diff'] = np.sum(dx > 0)
            f['n_negative_diff'] = np.sum(dx < 0)
            f['n_zero_diff'] = np.sum(dx == 0)

            # 5. Autocorrelazioni (lag 1-3)
            for lag in range(1, 4):
                ac = np.corrcoef(x[:-lag], x[lag:])[0, 1] if len(x) > lag else np.nan
                f[f'autocorr_lag{lag}'] = ac

            # 6. FFT (spettro frequenze)
            spectrum = np.abs(fft(x))
            half_spectrum = spectrum[:len(spectrum)//2]  # simmetrico
            f['fft_peak_freq'] = np.argmax(half_spectrum)
            f['fft_max'] = np.max(half_spectrum)
            f['fft_median'] = np.median(half_spectrum)
            f['fft_mean'] = np.mean(half_spectrum)

            features.append(f)

        return pd.DataFrame(features)


In [34]:

class ModelTrainer:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.final_model = None

    def plot_feature_importance(self, feature_names=None, top_n=20):
        """
        Plotta la feature importance del modello finale se disponibile.
        Args:
            feature_names (list or None): nomi delle feature, se disponibili.
            top_n (int): mostra solo le top_n feature.
        """
        import matplotlib.pyplot as plt
        importances = None

        if self.final_model is None:
            print("Nessun modello finale disponibile.")
            return

        # Tree-based models
        if hasattr(self.final_model, "feature_importances_"):
            importances = self.final_model.feature_importances_
        # Linear models
        elif hasattr(self.final_model, "coef_"):
            importances = self.final_model.coef_.ravel()
        else:
            print("Il modello non supporta la feature importance.")
            return

        if feature_names is None:
            feature_names = [f"f{i}" for i in range(len(importances))]

        # Ordina per importanza
        indices = importances.argsort()[::-1][:top_n]
        plt.figure(figsize=(10, 6))
        plt.barh(range(top_n), importances[indices][::-1], align="center")
        plt.yticks(range(top_n), [feature_names[i] for i in indices][::-1])
        plt.xlabel("Feature Importance")
        plt.title("Top Feature Importances")
        plt.tight_layout()
        plt.show()

    def train(self, model_search=True):
        os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"
        if model_search:
            scaler = MinMaxScaler()
            fit_data_scaled = scaler.fit_transform(self.data)
            clf = setup(data=fit_data_scaled,
                        target=self.labels,
                        pca=True,
                        pca_method='incremental',
                        use_gpu=True,
                        feature_selection=True
                        )
            best_models = compare_models(n_select=5)
            best_model = best_models[0]
            print(f"Il modello migliore è: {best_model}")
            tuned_model = tune_model(best_model,
                                    optimize='Accuracy',
                                    n_iter=10,
                                    search_library='optuna',
                                    search_algorithm='tpe',
                                    choose_better=True)
            print("Valutazione del modello ottimizzato:")
            evaluate_model(tuned_model)
            best_params = tuned_model.get_params()
            print("Migliori iperparametri trovati:")
            for param, value in best_params.items():
                print(f"{param}: {value}")
            self.final_model = finalize_model(tuned_model)
            save_model(self.final_model, 'best_binary_classification_model')
            # loaded_model = load_model('best_classification_model')

In [35]:
#ROOT_DIR = 'D:/GREAT/machine_learning/data/Rongowai/'


read_from_backup = True
if read_from_backup:
    #import polars as pl

    # Leggi i file parquet con polars
    fit_data_pl = pd.read_parquet('C:/Users/Alessandro/Desktop/great/rongowai/processed_data/binary_classification/fit_data_stratified_binary.parquet')
    labels_pl = pd.read_parquet('C:/Users/Alessandro/Desktop/great/rongowai/processed_data/binary_classification/labels_stratified_binary.parquet')

    # Trasforma in numpy array
    fit_data = fit_data_pl.to_numpy()
    labels = labels_pl['label'].to_numpy()
else:
    preprocessor = NetCDFPreprocessor(root_dir=ROOT_DIR)
    fit_data, labels = preprocessor.process_all_files(save=True)

In [36]:
preprocessor = NetCDFPreprocessor(root_dir='C:')
features_extractor = DDMFeatureExtractor()
ddm_features = features_extractor.create_ddm_features_MORE(fit_data)

Extracting DDM features: 100%|██████████| 1796779/1796779 [28:04<00:00, 1066.43it/s]


In [37]:

fit_data_with_features = np.hstack([fit_data, ddm_features.values])
fit_data_with_features.shape

(1796779, 252)

In [39]:
# Controlla infiniti o valori troppo grandi per float64
mask_finite = np.isfinite(fit_data_with_features).all(axis=1) & (np.abs(fit_data_with_features) < np.finfo(np.float64).max).all(axis=1)

fit_data_with_features_clean = fit_data_with_features[mask_finite]
labels_clean = labels[mask_finite]

In [ ]:
model_trainer = ModelTrainer(data=fit_data_with_features_clean, labels=labels_clean)
model_trainer.train(model_search=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Description,Value
0,Session id,777
1,Target,target
2,Target type,Binary
3,Original data shape,"(1780639, 253)"
4,Transformed data shape,"(1780639, 51)"
5,Transformed train set shape,"(1246447, 51)"
6,Transformed test set shape,"(534192, 51)"
7,Numeric features,252
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:45:43
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8542,0.9231,0.7515,0.7740,0.7626,0.6574,0.6576,218.0370
knn,K Neighbors Classifier,0.8232,0.8735,0.6545,0.7466,0.6975,0.5733,0.5758,239.9720
ada,Ada Boost Classifier,0.7891,0.8611,0.6123,0.6791,0.6439,0.4947,0.4960,441.7540
lr,Logistic Regression,0.7888,0.8571,0.6269,0.6729,0.6491,0.4983,0.4990,195.0520
dt,Decision Tree Classifier,0.7811,0.7451,0.6497,0.6483,0.6490,0.4900,0.4900,231.4210
svm,SVM - Linear Kernel,0.7745,0.8395,0.5802,0.6562,0.6158,0.4571,0.4589,100.4440
ridge,Ridge Classifier,0.7734,0.8435,0.5646,0.6592,0.6082,0.4502,0.4529,98.8900
qda,Quadratic Discriminant Analysis,0.6227,0.7962,0.9287,0.4490,0.6053,0.3195,0.3995,101.5380
nb,Naive Bayes,0.5587,0.7529,0.8767,0.4040,0.5531,0.2208,0.2885,99.0190


Processing:   0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
from pycaret.classification import load_model
import numpy as np
import matplotlib.pyplot as plt

# Carica il modello finale addestrato da pycaret
final_model = load_model('best_binary_classification_model')

# Ottieni la feature importance se disponibile

if hasattr(final_model, "feature_importances_"):
    importances = final_model.feature_importances_
elif hasattr(final_model, "coef_"):
    importances = final_model.coef_.ravel()
else:
    print("Il modello non supporta la feature importance.")
    importances = None

if importances is not None:
    # Usa i nomi delle colonne di fit_data_with_features se disponibili
    feature_names = [str(i) for i in range(fit_data_with_features.shape[1])]
    top_n = 20
    indices = np.argsort(importances)[::-1][:top_n]
    plt.figure(figsize=(10, 6))
    plt.barh(range(top_n), importances[indices][::-1], align="center")
    plt.yticks(range(top_n), [feature_names[i] for i in indices][::-1])
    plt.xlabel("Feature Importance")
    plt.title("Top Feature Importances")
    plt.tight_layout()
    plt.show()